In [13]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
import tensorflow as tf 
from tensorflow.keras.layers import *
import random
from cv2 import GaussianBlur
import numpy as np

In [8]:
def getDataset():
  (x_train, _), (x_test, _) = tf.keras.datasets.cifar10.load_data()
  normalize = lambda x: x/x.max()
  x_train = normalize(x_train)
  x_test = normalize(x_test)
  createBlurred = lambda arr: np.array([GaussianBlur(x, (5,5), ((random.random()*3)+1)) + (0.02*np.random.normal(0,1, (32, 32, 3))) for x in arr])
  return (createBlurred(x_train), x_train), (createBlurred(x_test), x_test)

In [9]:
data = getDataset()

In [18]:
input = Input(shape=(32,32,3),name='input')
x = Conv2D(64,kernel_size= (3,3), padding='same',name='conv2d_l1')(input)
x = Activation('relu',name='act_l1')(x)
for i in range(17):
    x = Conv2D(64, kernel_size=(3,3), padding='same',name='conv2d_'+str(i))(x)
    x = BatchNormalization(axis=-1,name='BN_'+str(i))(x)
    x = Activation('relu',name='act_'+str(i))(x)   
x = Conv2D(3, kernel_size=(3,3), padding='same',name='conv2d_l3')(x)
x = Subtract(name='subtract')([input, x])   

model=tf.keras.Model(input,x)

model.summary()
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
                  loss="mse",
                  metrics=['accuracy'])

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d_l1 (Conv2D)             (None, 32, 32, 64)   1792        ['input[0][0]']                  
                                                                                                  
 act_l1 (Activation)            (None, 32, 32, 64)   0           ['conv2d_l1[0][0]']              
                                                                                                  
 conv2d_0 (Conv2D)              (None, 32, 32, 64)   36928       ['act_l1[0][0]']                 
                                                                                            

In [ ]:
model.fit(data[0][0], data[0][1],
          batch_size = 16,
          epochs = 30)

Epoch 1/30
1928/3125 [=================>............] - ETA: 37:13 - loss: 0.0138 - accuracy: 0.7390

In [12]:
model.save_weights("17l-30e.h5")

In [17]:
model.evaluate(data[1][0], data[1][1])

313/313 [==============================] - 230s 731ms/step - loss: 0.0067 - accuracy: 0.7825


[0.006740631069988012, 0.7824873924255371]